In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import numpy as np

import util as u

from glob import glob
from tqdm import tqdm_notebook as tqdm

pd.set_option('display.max_columns', 2000)


In [36]:
#巡回先を作る
#枠とレースIDがあればマージできる？
result_infofile_list = glob("row_html/01/info_html/*") # 後々フォルダーが変わる
main_info_file = [x.replace("\\", "/") for x in result_infofile_list]

result_resultfile_list = glob("row_html/01/result_html/*") # 後々フォルダーが変わる
main_result_file = [x.replace("\\", "/") for x in result_resultfile_list]

target_info_list = [] #配列を空にする
target_result_list = [] #配列を空にする

match_up_info_list = [] #配列を空にする
match_up_result_list = [] #配列を空にする

is_file = os.path.isfile("match_up.txt")

"""
if is_file:
    with open("match_up.txt", "r") as tf:
        match_up_list = tf.read().split(',')[:-1]
    #途中から始めるためのコード書く
    target_txts_list = set(result_file) - set(match_up_list)
else:
    target_txts_list = result_file
"""

target_info_list = main_info_file[:1]
target_result_list = main_result_file[:1]

In [37]:
#データフレームを4個作る
main_info = pd.DataFrame(index=[])
main_info_dummy = pd.DataFrame(index=[])
main_result = pd.DataFrame(index=[])
main_result_dummy = pd.DataFrame(index=[])

dfs_all = pd.DataFrame(index=[])
dfs_dummy = pd.DataFrame(index=[])

In [38]:

# info_htmlの加工をする
# race_id、枠、選手ID、級別、体重(直前情報から取る)、調整重量、F数、L数、平均ST、モーター2連、モーター3連、初日進入、
# 初日ST、初日成績、～ 、最終日、展示タイム、チルト、プロペラ、部品交換

for target_info in tqdm(target_info_list):
    for i in range(1, 7):  #  番艇
        #  5月18日 ここを消しても良さそうなので消すが一応Gitにプッシュしときます
        target_info_day = target_info[-19:-5]
        Pit_No = f"{i:02}"
        
        #レース結果を保存したhtmlファイルとそのDF
        target_info_day = target_info[-19:-5]
        dfs = pd.read_html(target_info, encoding="utf-8")
        race_info1_df = dfs[0]
        
        #dfs[0]の整理
        df_player_info = u.split_inner_data(u.parse(race_info1_df), target_info_day)
        df_player_info = df_player_info.iloc[i-1]

        player_ID = df_player_info["pid"]

        #複勝率、三連率を計算
        player_win_rate = u.player_win_rate_cal(target_info_day, player_ID)
        se_player_win_rate = player_win_rate.iloc[0]

        #開催期間の艇番を表示
        df_bort_color = u.bort_color(target_info).iloc[i-1]
        #開催期間の艇番を表示
        df_open_grades = u.Open_Grades_sort(target_info).iloc[i-1]
        #パーツ交換を表示
        df_parts_info = u.parts_change(target_info).iloc[i-1]

        # パーツ交換を追加
        main_info_dummy = pd.concat([df_player_info, df_parts_info])
        #天気情報を追加
        main_info_dummy = pd.concat([main_info_dummy, u.weather_condition(target_info)])
        #勝率を追加
        main_info_dummy = pd.concat([main_info_dummy, se_player_win_rate])
        #開催期間の艇番を追加
        main_info_dummy = pd.concat([main_info_dummy, df_bort_color])
        #開催中のレース成績
        main_info_dummy = pd.concat([main_info_dummy, df_open_grades])
        #ダミーの情報をメイン情報に追加
        main_info = main_info.append(main_info_dummy, ignore_index=True)
        
        main_info["en"] = main_info["en"].astype(int)
        #with open("target_urls.txt", "a", encoding="utf-8") as file:
        #file.write( + ",")


C:\Users\rstta\AppData\Local\Temp/ipykernel_20008/1098757845.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for target_info in tqdm(target_info_list):


  0%|          | 0/1 [00:00<?, ?it/s]

In [54]:
display(main_info)

,rid,en,pid,class,branch,weight,F,L,st,mnww,mnwww,idx,Adj_weight,Open_t,tilt,pera,pis,shaft,gia,cab,electro,ring,silinda,calbo,temp,whether,win_s,win_a,temp_w,wave_h,q_Total_2,q_Total_3,h_Total_2,h_Total_3,q_Venue_2,q_Venue_3,h_Venue_2,h_Venue_3,bort_color_1_1,bort_color_1_2,bort_color_2_1,bort_color_2_2,bort_color_3_1,bort_color_3_2,bort_color_4_1,bort_color_4_2,bort_color_5_1,bort_color_5_2,bort_color_6_1,bort_color_6_2,bort_color_7_1,bort_color_7_2,insert_1_1,insert_1_2,insert_2_1,insert_2_2,insert_3_1,insert_3_2,insert_4_1,insert_4_2,insert_5_1,insert_5_2,insert_6_1,insert_6_2,insert_7_1,insert_7_2,start_timing_1_1,start_timing_1_2,start_timing_2_1,start_timing_2_2,start_timing_3_1,start_timing_3_2,start_timing_4_1,start_timing_4_2,start_timing_5_1,start_timing_5_2,start_timing_6_1,start_timing_6_2,start_timing_7_1,start_timing_7_2,rank_1_1,rank_1_2,rank_2_1,rank_2_2,rank_3_1,rank_3_2,rank_4_1,rank_4_2,rank_5_1,rank_5_2,rank_6_1,rank_6_2,rank_7_1,rank_7_2
0,01_20140102_01,1,3115,B1,群馬,50.0,0,0,0.22,0.00,-,01_20140102_01_01,0.0,6.85,-0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01_20140102_01,2,3560,B1,群馬,49.0,0,0,0.19,0.00,-,01_20140102_01_02,1.0,6.85,-0.5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01_20140102_01,3,3376,B1,群馬,48.5,0,0,0.24,14.20,-,01_20140102_01_03,1.5,6.87,-0.5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01_20140102_01,4,3532,A1,群馬,52.8,0,0,0.17,0.00,-,01_20140102_01_04,0.0,6.90,-0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01_20140102_01,5,2808,B2,群馬,51.4,0,0,0.29,0.00,-,01_20140102_01_05,0.0,6.97,-0.5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,01_20140102_01,6,4645,B1,群馬,50.4,0,0,0.16,66.60,-,01_20140102_01_06,0.0,6.85,-0.5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
"""
output_file = "main_info.csv"
main_info.to_csv(os.path.join(output_file), mode="a",header=True, index=False)
"""

'\noutput_file = "main_info.csv"\nmain_info.to_csv(os.path.join(output_file), mode="a",header=True, index=False)\n'

In [25]:
dfs1 = pd.read_html("row_html/01/info_html/01_20140115_06.html", encoding="utf-8")[2]
display(dfs1)

スタート展示                
      コース      並び      ST
0   1 .06   1 .06   1 .06
1   2 .08   2 .08   2 .08
2   3 .06   3 .06   3 .06
3  5 F.09  5 F.09  5 F.09
4   4 .10   4 .10   4 .10
5   6 .01   6 .01   6 .01

In [53]:
display(main_info.loc[4, "en"])

for i in range(6):
    x = dfs1.iloc[i,0].split(" ")
    x1 = int(x[0])
    if main_info.loc[4, "en"] == x1:
        print(i+1)
        print(x1)
        print(x[1])


5

4
5
F.09


In [11]:
for target_result in tqdm(target_result_list):
    dfs = pd.read_html(target_result, encoding="utf-8")[2]
    dfs = dfs.sort_values("枠").reset_index()
    dfs = dfs.drop("index", axis=1)

    x = dfs['着'].map({chr(ord('０')+i): i for i in range(1,7)})

    dfs["着"] = x
    waku = dfs["枠"]

    dfs["idx"] = f"{target_result[-19:-5]}_0" + waku.astype(str)

    main_result_dummy["idx"] = dfs["idx"]
    main_result_dummy["着"] = dfs["着"]
    
    main_result = main_result.append(main_result_dummy, ignore_index=True)


C:\Users\rstta\AppData\Local\Temp/ipykernel_8768/4152842243.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for target_result in tqdm(target_result_list):


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:

output_file = "main_result.csv"
main_result.to_csv(os.path.join(output_file), mode="a",header=False, index=False)


In [12]:
display(main_result)

,idx,着
0,01_20140102_01_01,3
1,01_20140102_01_02,4
2,01_20140102_01_03,5
3,01_20140102_01_04,2
4,01_20140102_01_05,6
5,01_20140102_01_06,1


In [19]:
display(main_info.head())
#display(main_info.corr().head())

,rid,en,pid,class,branch,weight,F,L,st,mnww,mnwww,idx,Adj_weight,Open_t,tilt,pera,pis,shaft,gia,cab,electro,ring,silinda,calbo,temp,whether,win_s,win_a,temp_w,wave_h,q_Total_2,q_Total_3,h_Total_2,h_Total_3,q_Venue_2,q_Venue_3,h_Venue_2,h_Venue_3,bort_color_1_1,bort_color_1_2,bort_color_2_1,bort_color_2_2,bort_color_3_1,bort_color_3_2,bort_color_4_1,bort_color_4_2,bort_color_5_1,bort_color_5_2,bort_color_6_1,bort_color_6_2,bort_color_7_1,bort_color_7_2,insert_1_1,insert_1_2,insert_2_1,insert_2_2,insert_3_1,insert_3_2,insert_4_1,insert_4_2,insert_5_1,insert_5_2,insert_6_1,insert_6_2,insert_7_1,insert_7_2,start_timing_1_1,start_timing_1_2,start_timing_2_1,start_timing_2_2,start_timing_3_1,start_timing_3_2,start_timing_4_1,start_timing_4_2,start_timing_5_1,start_timing_5_2,start_timing_6_1,start_timing_6_2,start_timing_7_1,start_timing_7_2,rank_1_1,rank_1_2,rank_2_1,rank_2_2,rank_3_1,rank_3_2,rank_4_1,rank_4_2,rank_5_1,rank_5_2,rank_6_1,rank_6_2,rank_7_1,rank_7_2
0,01_20140102_01,1,3115,B1,群馬,50.0,0,0,0.22,0.00,-,01_20140102_01_01,0.0,6.85,-0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01_20140102_01,2,3560,B1,群馬,49.0,0,0,0.19,0.00,-,01_20140102_01_02,1.0,6.85,-0.5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01_20140102_01,3,3376,B1,群馬,48.5,0,0,0.24,14.20,-,01_20140102_01_03,1.5,6.87,-0.5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01_20140102_01,4,3532,A1,群馬,52.8,0,0,0.17,0.00,-,01_20140102_01_04,0.0,6.90,-0.5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01_20140102_01,5,2808,B2,群馬,51.4,0,0,0.29,0.00,-,01_20140102_01_05,0.0,6.97,-0.5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
for main_info in tqdm(main_info_file):

    dfs = pd.read_html(main_info, encoding="utf-8")
    dfs = dfs[1][::4]
    
    output_file = "test.csv"
    dfs.to_csv(os.path.join(output_file), mode="a",header=False, index=False)


KeyboardInterrupt: 

In [21]:
for main_info in tqdm(main_info_file):

    dfs = pd.read_html(main_info, encoding="utf-8")
    dfs = dfs[1][::4]
    dfs = dfs.drop("前走成績", axis=1)
    dfs["race_id"] = main_info[22:-5]
    
    output_file = "test.csv"
    dfs.to_csv(os.path.join(output_file), mode="a",header=False, index=False)

C:\Users\rstta\AppData\Local\Temp/ipykernel_20008/630000531.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for main_info in tqdm(main_info_file):


  0%|          | 0/18910 [00:00<?, ?it/s]

c:\Users\rstta\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


KeyboardInterrupt: 

In [22]:
#結果から順位を取得する

dfs = pd.read_html(target_result_list[0], encoding="utf-8")[2]
dfs = dfs.sort_values("枠").reset_index()
dfs = dfs.drop("index", axis=1)

x = dfs['着'].map({chr(ord('０')+i): i for i in range(1,7)})

dfs["着"] = x
waku = dfs["枠"]

dfs["idx"] = f"{target_result_list[0][-19:-5]}_0" + waku.astype(str)

main_result_dummy["idx"] = dfs["idx"]
main_result_dummy["着"] = dfs["着"]
display(main_result_dummy)

,idx,着
0,01_20140102_01_01,3
1,01_20140102_01_02,4
2,01_20140102_01_03,5
3,01_20140102_01_04,2
4,01_20140102_01_05,6
5,01_20140102_01_06,1


In [23]:
display(main_info)

'row_html/01/info_html/01_20140115_06.html'

In [24]:
dfs = pd.read_html(main_info, encoding="utf-8")
display(dfs[2])

#コース、タイム、

スタート展示                
      コース      並び      ST
0   1 .06   1 .06   1 .06
1   2 .08   2 .08   2 .08
2   3 .06   3 .06   3 .06
3  5 F.09  5 F.09  5 F.09
4   4 .10   4 .10   4 .10
5   6 .01   6 .01   6 .01

### 現在開催されているデータを引っ張ってくる

### データが取れてないところからデータを取る処理を書く

### PyCaretってのを使ってみるよ

中国語

○海外アングラ系
○世界各国の風俗嬢のお昼ごはん密着系

○盆栽販売

インドで苦行僧密着

タイでラーメン屋開業
うどん屋

ドバイで物乞い
戦場Youtuber

捨て猫系Youtuber (猫アレルギー)

動物系Youtuber

ベトナム人彼女作る系おじさん

開拓系Youtuber(林を買って畑とかにする)


In [ ]:
ダイエット(95kgから65kg or 体脂肪10%切る)
インスタグラムと連動してやりたい

In [3]:
l = ['Alice', 'Bob', 'Charlie']

for i, name in enumerate(l, 42):
    print(i, name)

42 Alice
43 Bob
44 Charlie


In [ ]:
私 知り合い ヤクザ お塩